In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [35]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql"
article_df = pd.read_pickle(data_path)

In [5]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/physics.pql"
data = None
phy_categories = pd.read_pickle(data_path)

In [3]:
data = None
data = phy_categories.merge(article_df, 
                            how = 'inner',
                            on = 'ArticleID')

NameError: name 'phy_categories' is not defined

In [2]:
data = data[data.Label == "Physics, Applied"]
_data = data[(data.PubYear > 1990) & (data.PubYear < 2000) ]

NameError: name 'data' is not defined

In [1]:
_data['title_without_punct'] = _data['Title'].progress_apply(lambda x: 
                                                           str.lower(''.join(ch for ch in str(x) if category(ch)[0] != 'P')))
_data["title_without_stopwords"] = _data['title_without_punct'].progress_apply(lambda x: "".join([word for word in x.split() if word not in stopwords]))

NameError: name '_data' is not defined

In [9]:
len(data)

1031701

In [10]:
data.groupby('Label').size()

Label
Physics, Applied    1031701
dtype: int64

In [11]:
data.head()

,ArticleID,Type,Label,PubYear,Doctypes,Title,Journal,Journal Abbreviation ISO,title_without_punct,title_without_stopwords
2810,A1987K373700006,ASCA traditional subject,"Physics, Applied",1987,Article,APPLICATION OF AN ATMOSPHERIC-PRESSURE MICROWA...,PLASMA CHEMISTRY AND PLASMA PROCESSING,Plasma Chem. Plasma Process.,application of an atmosphericpressure microwav...,"[application, atmosphericpressure, microwave, ..."
2819,A1987H758700004,ASCA traditional subject,"Physics, Applied",1987,Article,THE EFFICACY OF HEATING LOW-PRESSURE H-2 IN A ...,PLASMA CHEMISTRY AND PLASMA PROCESSING,Plasma Chem. Plasma Process.,the efficacy of heating lowpressure h2 in a mi...,"[efficacy, heating, lowpressure, h2, microwave..."
2931,A1987H758700003,ASCA traditional subject,"Physics, Applied",1987,Article,DECOMPOSITION OF SICL4 AND DEPOSITION OF SI IN...,PLASMA CHEMISTRY AND PLASMA PROCESSING,Plasma Chem. Plasma Process.,decomposition of sicl4 and deposition of si in...,"[decomposition, sicl4, deposition, si, inducti..."
2942,A1987H758700001,ASCA traditional subject,"Physics, Applied",1987,Article,DOMINANT REACTION CHANNELS AND THE MECHANISM O...,PLASMA CHEMISTRY AND PLASMA PROCESSING,Plasma Chem. Plasma Process.,dominant reaction channels and the mechanism o...,"[dominant, reaction, channels, mechanism, sila..."
2976,A1987G608000006,ASCA traditional subject,"Physics, Applied",1987,Article,ION BOMBARDMENT-INDUCED MECHANICAL-STRESS IN P...,PLASMA CHEMISTRY AND PLASMA PROCESSING,Plasma Chem. Plasma Process.,ion bombardmentinduced mechanicalstress in pla...,"[ion, bombardmentinduced, mechanicalstress, pl..."


In [9]:
#_data = data[(data.PubYear > 1990) & (data.PubYear < 2000) ]

In [10]:
_data.PubYear.unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999])

In [11]:
data_grouped_by_year = _data.groupby('PubYear').agg({'title_without_stopwords': 'sum'}).reset_index()

In [ ]:
data_grouped_by_year["len"] = data_grouped_by_year["title_without_stopwords"].apply(lambda x: len(x))

In [39]:
# create a list of combining things
words_in_title = list(data_grouped_by_year["title_without_stopwords"])

all_words_until_past_year = []
new_words_in_the_year = []

top_words = []

all_words = []

import collections


for i in range(len(data_grouped_by_year)):
    
    words_in_the_year = words_in_title[i]
    
    counts = collections.Counter(words_in_the_year)
    high_freq = counts.most_common()[:1000]
    top_words.append(high_freq)

    new_words_in_the_year.append([word for word in high_freq if word not in all_words_until_past_year])
    all_words_until_past_year.extend(high_freq)
    all_words.extend(words_in_the_year)

In [31]:
all_words_counter = collections.Counter(all_words)
higest_freq = all_words_counter.most_common()[:50]

In [33]:
top_freq = pd.DataFrame(higest_freq, columns=["word","Freq"])

In [37]:
datatoplot = [go.Line(
            x=top_freq["word"],
            y=top_freq["Freq"]
    )]



layout = go.Layout(
    title='Top Frquency Words',
    
    xaxis=dict(
        title='Word',
        titlefont=dict(
            family='Lato',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            family='Lato',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



__What percentage of words are overlapping with the words from previous year?__

In [54]:
overlap_counts = []

for i in range(1, len(top_words)):
    overlap_count = len([word for word in top_words[i] if word in top_words[i-1]])/ len(top_words[i-1])    
    overlap_counts.append(overlap_count)


    
overlapping_word_count = pd.DataFrame([(year, overlap_counts[i]) for (i, year) in enumerate(range(1992, 2000))],
                                      columns=["Year","Overlap_Count"])

datatoplot = [go.Line(
            x=overlapping_word_count["Year"],
            y=overlapping_word_count["Overlap_Count"]
    )]
layout = go.Layout(
    title='Overlap Count of Words',
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Overlap Count',
        titlefont=dict(
            family='Lato',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [39]:
foo = data_grouped_by_year.rolling(7, on='title_without_stopwords',min_periods=0).sum()
data_grouped_by_year["rolled"] = data_grouped_by_year.title_without_stopwords.rolling(3, win_type ='triang').sum()

data_grouped_by_year["rolled_len"] = data_grouped_by_year["rolled"].apply(lambda x: len(x))
data_grouped_by_year

,PubYear,title_without_stopwords,len,rolled,rolled_len
0,1991,"[tridimensional, model, magnetostatic, case, c...",118557,"[tridimensional, model, magnetostatic, case, c...",118557
1,1992,"[counterflow, liquid, injection, plasma, synth...",114584,"[counterflow, liquid, injection, plasma, synth...",114584
2,1993,"[thermalanalysis, lowtemperature, groundlevel,...",127503,"[thermalanalysis, lowtemperature, groundlevel,...",127503
3,1994,"[fractal, characterization, 3d, surfaceroughne...",134213,"[fractal, characterization, 3d, surfaceroughne...",134213
4,1995,"[he3, nuclearspin, relaxation, cesium, coated,...",129310,"[he3, nuclearspin, relaxation, cesium, coated,...",129310
5,1996,"[radial, structure, kinetics, production, elec...",147185,"[radial, structure, kinetics, production, elec...",147185
6,1997,"[homogenization, materials, used, allfilm, met...",144867,"[homogenization, materials, used, allfilm, met...",144867
7,1998,"[nonlocal, electron, kinetics, pstriations, dc...",151542,"[nonlocal, electron, kinetics, pstriations, dc...",151542
8,1999,"[snox, thin, film, gas, sensors, prepared, pla...",152579,"[snox, thin, film, gas, sensors, prepared, pla...",152579
9,2000,"[numerical, modeling, hen2, capillary, surface...",155013,"[numerical, modeling, hen2, capillary, surface...",155013
